In [1]:
#import ipdb # alternative to pdb that works in jupyter notebook (pip3 install ipdb)
import os, subprocess, sys, re, time
from pathlib import Path
from pynq import Overlay
from pynq import allocate
from threading import Thread
from queue import Queue

from dma_receiver import DmaReceiver

BASE_DIR = Path('/home/xilinx/Django/pynq')

PATH = BASE_DIR / 'tools/make'

base = Overlay(str(BASE_DIR / 'dma_test.bit'))
#base = Overlay(str(BASE_DIR / 'onboard.bit'))


FIFO_SIZE = 26
input_buffer = allocate(shape=(FIFO_SIZE,), dtype='u8')
output_buffer = allocate(shape=(FIFO_SIZE,), dtype='u8')

dma_send = base.axi_dma_0.sendchannel
dma_rec = base.axi_dma_0.recvchannel

def print_dma_channel_status(channel, name='dma'):
    print(f'{name}.running =', channel.running)
    print(f'{name}.idle =', channel.idle)
    print(f'{name}.error =', channel.error)
    print(f'status =', hex(channel._mmio.read(channel._offset + 4)))
    
def print_dma_channels_status():
    print_dma_channel_status(dma_rec, name='dma_rec')
    print()
    print_dma_channel_status(dma_send, name='dma_send')

gpio_counts = base.axi_gpio_counts
gpio_flags = base.axi_gpio_flags

def print_flags():
    print('almost_empty =', gpio_flags[0].read())
    print('prog_empty =', gpio_flags[1].read())
    print('almost_full =', gpio_flags[2].read())
    print('prog_full =', gpio_flags[3].read())
    
write_count_channel = gpio_counts.channel1
read_count_channel = gpio_counts.channel2
def read_count():
    print('read count =', read_count_channel.read())
def write_count():
    print('write count =', write_count_channel.read())

input_buffers = [ allocate(shape=(16,), dtype='u8') for i in range(2) ]
output_buffer = allocate(shape=(16,), dtype='u8')

def print_buffers():
    #print('input_buffer =', ', '.join(f'{i:08X}' for i in input_buffer))
    #print('output_buffer =', ', '.join(f'{i:08X}' for i in output_buffer))
    print('input_buffer =', input_buffer)
    print('output_buffer =', output_buffer)

def print_stats():
    read_count()
    write_count()
    #print_flags()
    print()
    print_dma_channels_status()
    print_buffers()
    print()
    


# class DmaReceiver:
#     ACK = 0xFF
#     def __init__(self, dma, buffer_count=2, allocate_args=[], allocate_kwargs={'shape':(16,), 'dtype':'u8'}):
#         self.dma = dma
#         self.buffer_count = buffer_count
        
#         # item_size = the number of bytes in a single FIFO item
#         self.item_size = int( re.search('\d+$', allocate_kwargs.get('dtype') ).group(0) ) 
        
#         # input buffers
#         self.ibs = [allocate(*allocate_args, **allocate_kwargs) for _ in range(buffer_count)]
#         self.ib_queues = [Queue() for _ in range(buffer_count)]
        
#         self.rec_len_queue = Queue()
#         self.recv_thread = Thread(target=self.recv_thread, daemon=True)
#         self.recv_thread.start()
#         self.copying_thread = Thread(target=self.copying_thread, daemon=True)
#         self.copying_thread.start()
        
#         self.out_data_lock = Lock()
#         self.out_data = []
    
#     def recv_thread(self):
#         dma_rec = self.dma.recvchannel
#         active_ib = 0
#         while True:
#             ib = self.ibs[active_ib]
#             ib_queue = self.ib_queues[active_ib]
#             active_ib += 1
#             active_ib %= self.buffer_count
            
#             # if input buffer was used previously and received data wasn't copied to "self.out_data"
#             # then wait until that happens before issuing another transfer
#             while not ib_queue.empty():
#                 time.sleep(0.001)
#             dma_rec.transfer(ib)
#             dma_rec.wait()
#             bytes_count = self.get_last_recv_length()
#             ib_queue.put(1)
#             self.rec_len_queue.put(bytes_count)
    
#     def copying_thread(self):
#         active_ib = 0
#         while True:
#             ib = self.ibs[active_ib]
#             ib_queue = self.ib_queues[active_ib]
#             active_ib += 1
#             active_ib %= self.buffer_count
#             bytes_count = self.rec_len_queue.get(block=True) # blocking
#             data = list(ib[:bytes_count]) # make a copy of data
#             ib_queue.get(block=True) # allow another transfer for the same input buffer
#             with self.out_data_lock:
#                 self.out_data.extend(data)                      
        
#     def get_last_recv_length(self):
#         # dma = base.axi_dma_0
#         # 0x58 is the address of S2MM_length as shown in the table from:
#         # https://docs.xilinx.com/r/en-US/pg021_axi_dma/AXI-DMA-Register-Address-Map
#         return self.dma.mmio.read(0x58) // self.item_size
    
#     def data_available(self):
#         with self.out_data_lock:
#             return len(self.out_data) != 0
        
#     def get(self):
#         while not self.data_available():
#             time.sleep(0.001)
#         with self.out_data_lock:
#             data = list(self.out_data)
#             self.out_data = []
#             return data



In [2]:
base?

In [2]:
output_buffer[:] = [i+0xFFFAFF for i in range(1, len(output_buffer)+1)]


def original_speed_test(n=5):
    data = []
    for i in range(n):        
        dma_send.transfer(output_buffer)
        dma_rec.transfer(input_buffer)
        dma_send.wait()
        dma_rec.wait()
        data.extend(input_buffer.tolist())

def dma_receiver_speed_test(n=5):
    data = []
    for i in range(n):
        dma_send.transfer(output_buffer)
        dma_send.wait()        
        data.extend( dma_receiver.get() )

    
#     def receiver_thread():
#         dma_receiver = DmaReceiver(base.axi_dma_0, buffer_count=2)
#         for i in range(n):
#             print('receiver_thread i=',i)
#             data.extend( dma_receiver.get() )
#             print('receiver_thread after i=',i)
#     rt = Thread(target=receiver_thread, daemon=True)
#     rt.start()  
#     for i in range(n):
#         print('sending i=', i)
#         dma_send.transfer(output_buffer)
#         dma_send.wait()
#     rt.join()

start_time = time.time()
original_speed_test(n=500)
print('original took:', f'{time.time() - start_time}')

dma_receiver = DmaReceiver(base.axi_dma_0, buffer_count=4)

start_time = time.time()
dma_receiver_speed_test(n=500)
print('DmaReceiver took:', f'{time.time() - start_time}')

original took: 5.256540536880493
DmaReceiver took: 4.585379600524902


In [2]:
def receiver_thread(base):
    dma_receiver = DmaReceiver(base.axi_dma_0)
    while True:
        print( dma_receiver.get() )
    
rt = Thread(target=receiver_thread, args=[base], daemon=True)
rt.start()

In [2]:
def receiver_thread():
    dma_rec.transfer(input_buffer, nbytes=8)
    dma_rec.wait()
    
rt = Thread(target=receiver_thread, daemon=True)
rt.start()

In [4]:
output_buffer[:] = [i+0 for i in range(1, len(output_buffer)+1)]
dma_send.transfer(output_buffer)
dma_send.wait()

In [3]:
dma_rec.transfer(input_buffer)#, nbytes=8*26)
dma_rec.wait()

In [2]:
base.axi_dma_0.register_map

RegisterMap {
  MM2S_DMACR = Register(RS=1, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
  MM2S_DMASR = Register(Halted=0, Idle=0, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SGDecErr=0, IOC_Irq=0, Dly_Irq=0, Err_Irq=0, IRQThresholdSts=0, IRQDelaySts=0),
  MM2S_CURDESC = Register(Current_Descriptor_Pointer=0),
  MM2S_CURDESC_MSB = Register(Current_Descriptor_Pointer=0),
  MM2S_TAILDESC = Register(Tail_Descriptor_Pointer=0),
  MM2S_TAILDESC_MSB = Register(Tail_Descriptor_Pointer=0),
  MM2S_SA = Register(Source_Address=0),
  MM2S_SA_MSB = Register(Source_Address=0),
  MM2S_LENGTH = Register(Length=0),
  SG_CTL = Register(SG_CACHE=0, SG_USER=0),
  S2MM_DMACR = Register(RS=1, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
  S2MM_DMASR = Register(Halted=0, Idle=0, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SG

In [7]:
base.axi_dma_0.sendchannel.transferred

128

In [3]:
print_stats()

read count = 0
write count = 0
almost_empty = 1
prog_empty = 1
almost_full = 0
prog_full = 0

dma_rec.running = True
dma_rec.idle = False
dma_rec.error = False
status = 0x0

dma_send.running = True
dma_send.idle = False
dma_send.error = False
status = 0x0
input_buffer = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
output_buffer = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



In [3]:
base.axi_dma_0.register_map

RegisterMap {
  MM2S_DMACR = Register(RS=1, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
  MM2S_DMASR = Register(Halted=0, Idle=0, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SGDecErr=0, IOC_Irq=0, Dly_Irq=0, Err_Irq=0, IRQThresholdSts=0, IRQDelaySts=0),
  MM2S_CURDESC = Register(Current_Descriptor_Pointer=0),
  MM2S_CURDESC_MSB = Register(Current_Descriptor_Pointer=0),
  MM2S_TAILDESC = Register(Tail_Descriptor_Pointer=0),
  MM2S_TAILDESC_MSB = Register(Tail_Descriptor_Pointer=0),
  MM2S_SA = Register(Source_Address=0),
  MM2S_SA_MSB = Register(Source_Address=0),
  MM2S_LENGTH = Register(Length=0),
  SG_CTL = Register(SG_CACHE=0, SG_USER=0),
  S2MM_DMACR = Register(RS=1, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
  S2MM_DMASR = Register(Halted=0, Idle=0, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SG

In [5]:
base.axi_dma_0.register_map

RegisterMap {
  MM2S_DMACR = Register(RS=0, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
  MM2S_DMASR = Register(Halted=1, Idle=0, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SGDecErr=0, IOC_Irq=1, Dly_Irq=0, Err_Irq=0, IRQThresholdSts=0, IRQDelaySts=0),
  MM2S_CURDESC = Register(Current_Descriptor_Pointer=0),
  MM2S_CURDESC_MSB = Register(Current_Descriptor_Pointer=0),
  MM2S_TAILDESC = Register(Tail_Descriptor_Pointer=0),
  MM2S_TAILDESC_MSB = Register(Tail_Descriptor_Pointer=0),
  MM2S_SA = Register(Source_Address=1027735552),
  MM2S_SA_MSB = Register(Source_Address=0),
  MM2S_LENGTH = Register(Length=128),
  SG_CTL = Register(SG_CACHE=0, SG_USER=0),
  S2MM_DMACR = Register(RS=0, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
  S2MM_DMASR = Register(Halted=1, Idle=0, SGIncld=0, DMAIntErr=1, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGS